# 🏎️ Model 1: Random Forest Baseline

**Process:**
1. Train baseline Random Forest
2. Define hyperparameter search space
3. Use RandomizedSearchCV to find optimal parameters
4. Compare baseline vs optimized model
5. Visualize performance improvements
6. Save the best model

## Step 1 : Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import joblib
import os
from pathlib import Path
from datetime import datetime

import warnings
warnings.filterwarnings('ignore')

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import (
    mean_absolute_error,
    mean_squared_error,
    r2_score,
    make_scorer
)

sns.set_style("darkgrid")
plt.rcParams['figure.figsize'] = (12,6)
plt.rcParams['font.size'] = 10